In [10]:
from tensorflow.keras.preprocessing import image as keras_image
from tensorflow.keras.models import load_model
import numpy as np


# Function to load and preprocess an image for prediction
def load_and_preprocess_image(image_path, target_size):
    img = keras_image.load_img(image_path, target_size=target_size)
    img_array = keras_image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Normalize pixel values between 0 and 1
    return img_array

# Load a sample image for prediction
sample_image_path = r'ultrasound_data\malignant\malignant (1).png'  # Replace with your image path
target_size = (150, 150)  # Same as the target size used in the model

preprocessed_image = load_and_preprocess_image(sample_image_path, target_size)

# Load the saved best model
saved_model_path = 'best_model_1.h5'  # Replace with your saved model path
loaded_model = load_model(saved_model_path)

# Define class indices based on your training classes
class_indices = {'benign': 0, 'malignant': 1, 'normal': 2}  # Update with your class indices

# Make predictions
predictions = loaded_model.predict(preprocessed_image)
predicted_class_index = np.argmax(predictions)  # Get the index of the predicted class
predicted_class = list(class_indices.keys())[list(class_indices.values()).index(predicted_class_index)]

print(f"The predicted class is: {predicted_class}")



1/1 [==============================] - 0s 237ms/step
The predicted class is: normal


In [14]:
import os

# Directory containing the images to predict
image_directory = r'ultrasound_data\malignant'  # Replace with your image directory

# Iterate through the images in the directory
for filename in os.listdir(image_directory):
    if filename.endswith('.png'):  # Consider only PNG files, you can adjust this based on your file format
        image_path = os.path.join(image_directory, filename)
        preprocessed_image = load_and_preprocess_image(image_path, target_size)

        # Make predictions
        predictions = loaded_model.predict(preprocessed_image)
        predicted_class_index = np.argmax(predictions)
        predicted_class = list(class_indices.keys())[list(class_indices.values()).index(predicted_class_index)]

        print(f"Image: {filename}, Predicted class: {predicted_class}")


1/1 [==============================] - 0s 374ms/step
Image: malignant (1).png, Predicted class: normal
1/1 [==============================] - 0s 459ms/step
Image: malignant (1)_mask.png, Predicted class: malignant
1/1 [==============================] - 0s 329ms/step
Image: malignant (10).png, Predicted class: normal
1/1 [==============================] - 0s 281ms/step
Image: malignant (10)_mask.png, Predicted class: normal
1/1 [==============================] - 0s 354ms/step
Image: malignant (100).png, Predicted class: normal
1/1 [==============================] - 0s 343ms/step
Image: malignant (100)_mask.png, Predicted class: benign
1/1 [==============================] - 0s 288ms/step
Image: malignant (101).png, Predicted class: normal
1/1 [==============================] - 0s 347ms/step
Image: malignant (101)_mask.png, Predicted class: normal


In [15]:
import os
import csv

# Directory containing the images to predict
image_directory = r'ultrasound_data copy 2\malignant'  # Replace with your image directory

# Create and open a CSV file for writing predictions
csv_file_path = 'predictions.csv'
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Filename', 'Predicted_Class'])  # Writing header row

    # Iterate through the images in the directory
    for filename in os.listdir(image_directory):
        if filename.endswith('.png'):  # Consider only PNG files, you can adjust this based on your file format
            image_path = os.path.join(image_directory, filename)
            preprocessed_image = load_and_preprocess_image(image_path, target_size)

            # Make predictions
            predictions = loaded_model.predict(preprocessed_image)
            predicted_class_index = np.argmax(predictions)
            predicted_class = list(class_indices.keys())[list(class_indices.values()).index(predicted_class_index)]

            # Write filename and predicted class to CSV
            writer.writerow([filename, predicted_class])

print(f"Predictions written to {csv_file_path}")


1/1 [==============================] - 0s 88ms/step
Predictions written to predictions.csv


In [17]:
import os
import csv
import shutil

# Directory containing the images to predict
image_directory = r'ultrasound_data copy 2\normal'  # Replace with your image directory

# Directory to save images classified as 'malignant'
output_directory = 'normal_detected'  # Replace with your desired output directory

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Create and open a CSV file for writing predictions
csv_file_path = 'predictions.csv'
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Filename', 'Predicted_Class'])  # Writing header row

    # Iterate through the images in the directory
    for filename in os.listdir(image_directory):
        if filename.endswith('.png'):  # Consider only PNG files, you can adjust this based on your file format
            image_path = os.path.join(image_directory, filename)
            preprocessed_image = load_and_preprocess_image(image_path, target_size)

            # Make predictions
            predictions = loaded_model.predict(preprocessed_image)
            predicted_class_index = np.argmax(predictions)
            predicted_class = list(class_indices.keys())[list(class_indices.values()).index(predicted_class_index)]

            # Write filename and predicted class to CSV
            writer.writerow([filename, predicted_class])

            # Check if the predicted class is 'malignant' and move the file to the output directory
            if predicted_class == 'normal':
                shutil.copy(image_path, os.path.join(output_directory, filename))

print(f"Predictions written to {csv_file_path}")
print(f"Images classified as 'normal' saved in '{output_directory}'")


1/1 [==============================] - 0s 80ms/step
Predictions written to predictions.csv
Images classified as 'normal' saved in 'normal_detected'


In [19]:
import os
from collections import defaultdict

def count_images_in_directory(parent_dir):
    image_count = defaultdict(int)

    for root, dirs, files in os.walk(parent_dir):
        for file in files:
            _, extension = os.path.splitext(file)
            if extension.lower() in ['.png', '.jpg', '.jpeg', '.gif', '.bmp']:
                image_count[root] += 1

    return image_count

# Replace 'parent_directory_path' with your directory path
parent_directory_path = r'detect'

image_counts = count_images_in_directory(parent_directory_path)

for directory, count in image_counts.items():
    print(f"Directory: {directory} - Image Count: {count}")


Directory: detect\benign_detected - Image Count: 220
Directory: detect\malignant_detected - Image Count: 65
Directory: detect\normal_detected - Image Count: 84
